# Database intro 

In [1]:
# importing the library
from peewee import *

In [2]:
# defining our database and giving it a name
db = SqliteDatabase('maryland_athletics.db')

# define the base model 
# this connects every model we define to the same database.
class BaseModel(Model):
    class Meta:
        database = db

## Let's create our first table!

In [3]:
class Team(BaseModel):
    name = CharField(unique=True)
    sport = CharField()
    season = CharField()

In [4]:
# connecting to our maryland_athletics.db database
db.connect()
# telling it to create the team table, which we've defined.
db.create_tables([Team])

In [5]:
# Let's take a look at this table. Is there anything in it?
for team in Team.select():
    print(team.name)

In [6]:
# insert data into our table 
wbb_25, created = Team.get_or_create(
    name="Maryland Women's Basketball",
    defaults={
        "sport": "Women's Basketball",
        "season": "2025"
    }
)

msoc_25, created = Team.get_or_create(
    name="Maryland Men's Soccer",
    defaults={
        "sport": "Men's Soccer",
        "season": "2025"
    }
)

fb_25, created = Team.get_or_create(
    name="Maryland Football",
    defaults={
        "sport": "Football",
        "season": "2025"
    }
)

In [7]:
for team in Team.select():
    print(team.name)

Maryland Women's Basketball
Maryland Men's Soccer
Maryland Football


In [8]:
class Athlete(BaseModel):
    name = CharField()
    team = ForeignKeyField(Team, backref='athletes')
    athlete_year = CharField()
    athlete_position = CharField()

db.create_tables([Athlete])

In [9]:
malik_washington, created = Athlete.get_or_create(
    name="Malik Washington",
    defaults={
        "team": fb_25,
        "athlete_year": "Junior",
        "athlete_position": "Wide Receiver"
    }
)

In [10]:
for team in Team.select():
    print(team.name)

Maryland Women's Basketball
Maryland Men's Soccer
Maryland Football


In [11]:
class Game(BaseModel):
    date = DateField()
    opponent = CharField()
    home_away = CharField()     
    team = ForeignKeyField(Team, backref='games')
    

# stats are sport-agnostic — null fields just won't apply to every sport
class Stat(BaseModel):
    game = ForeignKeyField(Game, backref='stats')
    athlete = ForeignKeyField(Athlete, backref='stats')
    minutes_played = IntegerField(null=True)
    points = IntegerField(null=True)
    rebounds = IntegerField(null=True)
    assists = IntegerField(null=True)
    steals = IntegerField(null=True)
    turnovers = IntegerField(null=True)
    blocks = IntegerField(null=True)
    goals = IntegerField(null=True)
    shots = IntegerField(null=True)
    saves = IntegerField(null=True)

db.create_tables([Game, Stat])

In [12]:
# close the database connection when we're done
db.close()

True

## Now let's go over adding data using a spreadsheet and turning that into a database!


In [13]:
# read the spreadsheet
import pandas as pd
mcap_24 = pd.read_csv('data/data-9Tdi0.csv')

In [14]:
mcap_24

,District,School,Assessment,"Proficient, 2022","Proficient, 2023","Proficient, 2024",Percentage point change 2022-'23,Percentage point change 2023-'24
0,Montgomery,A. Mario Loiederman Middle,Mathematics 6,6,9.9,9.5,3.9,-0.4
1,Montgomery,A. Mario Loiederman Middle,Algebra 1,5,5,14,NaN,NaN
2,Montgomery,A. Mario Loiederman Middle,Geometry,5.7,9.3,14.3,3.6,5.0
3,Montgomery,A. Mario Loiederman Middle,English Language Arts 8,35.2,28.8,31.4,-6.4,2.6
4,Montgomery,A. Mario Loiederman Middle,English Language Arts 6,32.6,35.8,37.7,3.2,1.9
...,...,...,...,...,...,...,...,...
9086,Harford,Youths Benefit Elementary,Mathematics 4,47.8,52.4,56,4.6,3.6
9087,Harford,Youths Benefit Elementary,Mathematics 3,66.1,68,56,1.9,-12.0
9088,Harford,Youths Benefit Elementary,English Language Arts 5,58.8,59.2,67.6,0.4,8.4
9089,Harford,Youths Benefit Elementary,English Language Arts 3,66.9,70.7,67.9,3.8,-2.8


In [15]:
# get rid of the rows where district is called "seed" — these are just the average proficiency values for the state, not actual districts
mcap_24 = mcap_24[~mcap_24["District"].str.lower().eq("seed")]

## Clean up the data

In [16]:
# make the data long
long_df = mcap_24.melt(
    id_vars=["District", "School", "Assessment"],
    value_vars=[
        "Proficient, 2022",
        "Proficient, 2023",
        "Proficient, 2024"
    ],
    var_name="year",
    value_name="proficient_percent"
)

In [17]:
long_df

,District,School,Assessment,year,proficient_percent
0,Montgomery,A. Mario Loiederman Middle,Mathematics 6,"Proficient, 2022",6
1,Montgomery,A. Mario Loiederman Middle,Algebra 1,"Proficient, 2022",5
2,Montgomery,A. Mario Loiederman Middle,Geometry,"Proficient, 2022",5.7
3,Montgomery,A. Mario Loiederman Middle,English Language Arts 8,"Proficient, 2022",35.2
4,Montgomery,A. Mario Loiederman Middle,English Language Arts 6,"Proficient, 2022",32.6
...,...,...,...,...,...
27238,Harford,Youths Benefit Elementary,Mathematics 4,"Proficient, 2024",56
27239,Harford,Youths Benefit Elementary,Mathematics 3,"Proficient, 2024",56
27240,Harford,Youths Benefit Elementary,English Language Arts 5,"Proficient, 2024",67.6
27241,Harford,Youths Benefit Elementary,English Language Arts 3,"Proficient, 2024",67.9


In [18]:
# create a new year column based on the old year column
long_df["year"] = long_df["year"].astype(str).str.extract(r'(\d{4})').astype(int)

In [19]:
long_df["proficient_percent"] = (
    long_df["proficient_percent"]
    .replace("*", None)
    .astype(float)
)

In [20]:
long_df

,District,School,Assessment,year,proficient_percent
0,Montgomery,A. Mario Loiederman Middle,Mathematics 6,2022,6.0
1,Montgomery,A. Mario Loiederman Middle,Algebra 1,2022,5.0
2,Montgomery,A. Mario Loiederman Middle,Geometry,2022,5.7
3,Montgomery,A. Mario Loiederman Middle,English Language Arts 8,2022,35.2
4,Montgomery,A. Mario Loiederman Middle,English Language Arts 6,2022,32.6
...,...,...,...,...,...
27238,Harford,Youths Benefit Elementary,Mathematics 4,2024,56.0
27239,Harford,Youths Benefit Elementary,Mathematics 3,2024,56.0
27240,Harford,Youths Benefit Elementary,English Language Arts 5,2024,67.6
27241,Harford,Youths Benefit Elementary,English Language Arts 3,2024,67.9


In [21]:
# drop rows with missing proficiency values
long_df = long_df.dropna(subset=["proficient_percent"])

In [22]:
# create a new year column based on the old year column
long_df["year"] = long_df["year"].astype(str).str.extract(r'(\d{4})').astype(int)

In [23]:
# create a new database for the mcap data
db = SqliteDatabase("mcap.db")

# define the schemas for all of our tables!
class BaseModel(Model):
    class Meta:
        database = db
# district table
class District(BaseModel):
    name = CharField(unique=True)
# school table
class School(BaseModel):
    name = CharField()
    district = ForeignKeyField(District, backref="schools")

    class Meta:
        indexes = (
            (("name", "district"), True),
        )
# assessment table
class Assessment(BaseModel):
    name = CharField(unique=True)
# result table
class Result(BaseModel):
    school = ForeignKeyField(School, backref="results")
    assessment = ForeignKeyField(Assessment, backref="results")
    year = IntegerField()
    proficient_percent = FloatField()

    class Meta:
        indexes = (
            (("school", "assessment", "year"), True),
        )

In [24]:
# create the tables
db.connect()
db.create_tables([District, School, Assessment, Result])

In [25]:
# insert districts
for district_name in long_df["District"].unique():
    District.get_or_create(name=district_name)
# insert schools
for _, row in long_df[["School", "District"]].drop_duplicates().iterrows():
    district = District.get(District.name == row["District"])

    School.get_or_create(
        name=row["School"],
        district=district
    )
# insert assessments
for assessment_name in long_df["Assessment"].unique():
    Assessment.get_or_create(name=assessment_name)
# insert results
for _, row in long_df.iterrows():
    school = School.get(School.name == row["School"])
    assessment = Assessment.get(Assessment.name == row["Assessment"])

    Result.get_or_create(
        school=school,
        assessment=assessment,
        year=row["year"],
        defaults={"proficient_percent": row["proficient_percent"]}
    )